In [47]:
import pandas as pd
import numpy as np

## Get Data from Yahoo

In [2]:
from findatapy.market import Market, MarketDataRequest, MarketDataGenerator
from findatapy.util import DataConstants, LoggerManager

In [6]:
def load_decade(start_date = "decade", tickers = 'SPY'):
    logger = LoggerManager.getLogger(__name__)

    market = Market(market_data_generator=MarketDataGenerator())

    DataConstants.market_thread_technique = 'thread'

    # load S&P 500 ticker via wikipedia
    # snp = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    # tickers = snp[0]['Symbol'].to_list()
    
    # download equities data from Yahoo
    md_request = MarketDataRequest(
        start_date=start_date,
        data_source='yahoo',  # use Bloomberg as data source
        tickers=tickers,  # ticker (findatapy)
        fields=['close', 'open', 'high', 'low', 'volume'],  # which fields to download
        vendor_tickers=tickers,  # ticker (Yahoo)
        vendor_fields=['Close', 'Open', 'High', 'Low', 'Volume'])  # which Bloomberg fields to download)


    logger.info("Loading data with threading")

    df = market.fetch_market(md_request)

    logger.info("Loading data with multiprocessing")

    DataConstants.market_thread_technique = 'multiprocessing'

    df = market.fetch_market(md_request)

    logger.info("Loaded data with multiprocessing")

    return df

In [39]:
from findatapy.util import SwimPool; SwimPool()

spy_df = load_decade(start_date = '01 Jan 2019')

2021-02-07 14:57:12,646 - __main__ - INFO - Loading data with threading
2021-02-07 14:57:12,648 - findatapy.market.datavendorweb - INFO - Request Yahoo data
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
2021-02-07 14:57:13,078 - findatapy.market.datavendorweb - INFO - Completed request from Yahoo.
2021-02-07 14:57:13,086 - findatapy.market.ioengine - WARNING - Couldn't push MarketDataRequest_735__abstract_curve_key-None__base_depos_currencies-EUR_GBP_AUD_NZD_USD_CAD_CHF_NOK_SEK_JPY__base_depos_tenor-ON_TN_SN_1W_2W_3W_1M_2M_3M_4M_6M_9M_1Y_2Y_3Y_5Y__category-None__category_key-backtest_default-cat_yahoo_daily_NYC__cut-NYC__data_source-yahoo__environment-backtest__expiry_date-NaT__fields-close_open_high_low_volume__finish_date-2021-02-07 06:21:55_064435__freq-daily__freq_mult-1__fx_forwards_tenor-ON_TN_SN_1W_2W_3W_1M_2M_3M_4M_6M_9M_1Y_2Y_3Y_5Y__fx_vol_part-V_25R_10R_25B_10B__fx_vol_tenor-ON_1W_2W_3

In [40]:
spy_df = spy_df.rename(columns={"SPY.close":"Close", "SPY.open":"Open", "SPY.high":"High", "SPY.low":"Low", "SPY.volume":"Volume"})
spy_df.head()

,Close,Open,High,Low,Volume
Date,,,,,
2019-01-02,250.179993,245.979996,251.210007,245.949997,126925200.0
2019-01-03,244.210007,248.229996,248.570007,243.669998,144140704.0
2019-01-04,252.389999,247.589996,253.110001,247.169998,142628800.0
2019-01-07,254.380005,252.690002,255.949997,251.690002,103139104.0
2019-01-08,256.769989,256.820007,257.309998,254.000000,102512600.0


In [48]:
for col in list(spy_df.columns):
    spy_df[col] = spy_df[col].astype(np.float64)

### load SMA from TA-lib

In [50]:
from talib import abstract
# directly import sma
SMA = abstract.SMA

### Backtest with SMA

In [53]:
from backtesting import Strategy
from backtesting.lib import crossover


class SmaCross(Strategy):
    # Define the a MA lags as *class variables*
    # for later optimization
    n = 25
    
    def init(self):
        # Precompute the moving averages
        self.sma = self.I(SMA, self.data.Close, self.n)
    
    def next(self):
        # If closing price crosses above sma, close any existing
        # short trades, and buy the asset
        if crossover(self.data.Close, self.sma):
            self.position.close()
            self.buy()

        # Else, if closing price crosses below sma, close any existing
        # long trades, and sell the asset
        elif crossover(self.sma, self.data.Close):
            self.position.close()
            self.sell()

In [54]:
from backtesting import Backtest

bt = Backtest(spy_df, SmaCross, cash=10_000, commission=.002)
stats = bt.run()
stats

Start                     2019-01-02 00:00:00
End                       2021-02-05 00:00:00
Duration                    765 days 00:00:00
Exposure Time [%]                     91.4934
Equity Final [$]                      12957.5
Equity Peak [$]                       14012.3
Return [%]                            29.5746
Buy & Hold Return [%]                 54.9724
Return (Ann.) [%]                     13.1361
Volatility (Ann.) [%]                 24.2084
Sharpe Ratio                         0.542627
Sortino Ratio                        0.926414
Calmar Ratio                         0.873284
Max. Drawdown [%]                    -15.0422
Avg. Drawdown [%]                    -3.74128
Max. Drawdown Duration      157 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   34
Win Rate [%]                          41.1765
Best Trade [%]                         15.182
Worst Trade [%]                      -3.75646
Avg. Trade [%]                    

## With Stochastic Indicator

https://www.investopedia.com/articles/technical/073001.asp

In [27]:
# directly import stochastic
STOCH = abstract.STOCH
# uses high, low, close (default) to compute K line and D line
slowk, slowd = STOCH(high=spy_df['high'], low=spy_df['low'], close=spy_df['close'], fastk_period=5, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)